In [1]:
import logging
import time
logger = logging.getLogger('test')

def connect_with_retry(connector, retry_n_times, retry_threshold = 5):
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                f'{e}: 새로운 연결 시도 {retry_threshold}'
            )
            time.sleep(retry_threshold)

    exc = ConnectionError(f'{retry_n_times} 번째 연결 시도 실패')
    logger.exception(exc)
    raise exc

class DataTransport:
    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None
    
    def deliver_event(self, event):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(event)

    def send(self, event):
        try:
            self.connection.send(event.decode())
        except ValueError as e:
            logger.error('%r 잘못된 데이터 포함: %s', event, e)
            raise



## 올바른 추상화 단계에서 예외 처리
- 예외처리는 캡슐화를 약화시킨다, 필요한 예외처리만 하고, 필요한 위치에서 예외처리해야 한다
- `deliver_event의` 예외처리를 `connect와` `send` 위치로 분리

In [4]:
class InternalDataError(Exception):
    def __init__(self, msg):
        self._msg = msg

    def __str__(self):
        return f'Custom Error: {self._msg}'

def process(data_dict: dict, record_id: str):
    try:
        return data_dict[record_id]
    except KeyError as e:
        raise InternalDataError('Record not present') from e
        

process({0:1, 1:2}, 'hi')  

InternalDataError: Custom Error: Record not present

## 원본예외를 포함시키자
- Custom error 클래스로 예외 처리를 하더라도, 원본 예외를 확인할 수 있도록 하자